In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import spearmanr
from scipy.stats import norm

Exploration of the time series of Confluence 68:

Annual maxima sets starting the block maxima the 1st of january

Annual maxima sets starting the block maxima in the dry season

Time series of the minimum and maximun peaks

In [ ]:
#See MetFunctions AM
def AM(conf_):
    """
    Annual Maxima sets
    ds: dictionary with all the Time series per station 
    conf: configuration of the confluences
    c: confluence to be analyzed
    w: window of the multi-day events
    Returns: ds_
    """
    #create the Pandas DataFrame
#     conf_ = pd.DataFrame(columns=['year', 'month', 'day', 'P_S1', 'P_S2'])
#     conf_['Q_S1'] = ds[conf.HVB_Q1[c]]
#     conf_['Q_S2'] = ds[conf.HVB_Q2[c]]
#     conf_['year'] = ds['YEAR']
#     conf_['month'] = ds['MONTH']
#     conf_['day'] = ds['DAY']
    
    #Delete the rows with values lower than 0:
    conf_1 = conf_[~(conf_['Q_S1'] <= 0)]   
    conf_1 = conf_[~(conf_['Q_S2'] <= 0)]   
#     conf_1['P_S1'].loc[0] = 0      --> before I was just making the 1st value = to 0
#     conf_1['P_S2'].loc[0] = 0
    
    #1 Rolling 5 days
#     if w == 5:  #Check if this makes sense..  
#         conf_1['P_S1rol'] = conf_1['P_S1'].rolling(window=w).mean()
#         conf_1['P_S2rol'] = conf_1['P_S2'].rolling(window=w).mean()
#         conf_1['Conf'] = conf_1.P_S1rol + conf_1.P_S2rol
#     elif w == 1:
#     conf_1['P_S1rol'] = conf_1['Q_S1']
#     conf_1['P_S2rol'] = conf_1['Q_S2']
    conf_1['Conf'] = conf_1.Q_S1 + conf_1.Q_S2
    
    #2 Extreme selection sets
#     conf_1.index = conf_1.index.astype(int)
#     conf_1['Time'] = pd.to_datetime(conf_1.index, unit='d')
#     conf_1['year']=conf_1.Time.dt.year
#     conf_1['month']=conf_1.Time.dt.month
    
    
    
    #Seasonal averages
    conditions = [
        (conf_1.month==1)|(conf_1.month==2)|(conf_1.month==12),
        (conf_1.month==3)|(conf_1.month==4)|(conf_1.month==5),
        (conf_1.month==6)|(conf_1.month==7)|(conf_1.month==8),
        (conf_1.month==9)|(conf_1.month==10)|(conf_1.month==11)]

    choices = ['DJF', 'MAM', 'JJA', 'SON']
    conf_1['season'] = np.select(conditions, choices, default='ERROR')
    
    savg_S1 = conf_1.groupby(['season'])['Q_S1'].mean()
    savg_S2 = conf_1.groupby(['season'])['Q_S2'].mean()
    savg_C = conf_1.groupby(['season'])['Conf'].mean()

    
    numyears = conf_1.year.iloc[-1] - conf_1.year.iloc[0] + 1
    firstyear = conf_1.year.iloc[0]


    am1 = pd.DataFrame() #columns=['Time', 'year', 'AccumP_rol0', 'AccumP_rol1']
    am2 = pd.DataFrame()
    am3 = pd.DataFrame()
    for i in range(numyears):
        am1 = am1.append(conf_1.loc[conf_1.Q_S1[conf_1.year == firstyear + i].idxmax()])#, ignore_index=True)
        am2 = am2.append(conf_1.loc[conf_1.Q_S2[conf_1.year == firstyear + i].idxmax()])
        am3 = am3.append(conf_1.loc[conf_1.Conf[conf_1.year == firstyear + i].idxmax()])        

    return am1, am2, am3, savg_S1, savg_S2, savg_C

In [ ]:
conf = pd.read_csv(r"C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Notebooks\HBV\Confluences_ok2.csv", delimiter=';')

## HID_02
### AM files & Savg
2000 peaks per file

In [ ]:
for j in range(0, 25):
    conf_ = pd.read_csv(r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_TS\C68_TS%d.csv" %j, delimiter=';')
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_AM\C68_AM%d_" %j
    am1_, am2_, am3_, savg_S1_, savg_S2_, savg_C_ = AM(conf_)
    am1_.to_pickle(sv + "AM1.pkl")
    am2_.to_pickle(sv + "AM2.pkl")
    am3_.to_pickle(sv + "AM3.pkl")
    savg_S1_.to_pickle(sv + "savg_S1.pkl")
    savg_S2_.to_pickle(sv + "savg_S2.pkl")
    savg_C_.to_pickle(sv + "savg_C.pkl")

In [ ]:
# THE NEW FILES FROM 01_TSfiles
TS = pd.DataFrame()
for j in range(0, 25):
    conf_ = pd.read_csv(r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\01_TSfiles\TS%d_C68.csv" %j, delimiter=';')
    TS = pd.concat([TS, conf_])
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\01_TSonefile"
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_AM\C68_AM%d_" %j


In [ ]:
conditions = [
        (TS.month==1)|(TS.month==2)|(TS.month==3)|(TS.month==4)|(TS.month==5)|(TS.month==6)|(TS.month==7)|(TS.month==8),
        (TS.month==9)|(TS.month==10)|(TS.month==11)|(TS.month==12)]

choices = [TS['year'], TS['year'] + 1]
TS['yearAM'] = np.select(conditions, choices, default='ERROR')

In [ ]:
TS_ = TS[~(TS['Q_S1'] < 0)]
TS_ = TS[~(TS['Q_S2'] < 0)]

In [ ]:
TS_['Q_C'] = TS_['Q_S1'] + TS_['Q_S2']

In [ ]:
TS_.index = np.arange(len(TS_))

In [ ]:
AM1_try = TS_.groupby(['yearAM'])['Q_S1'].idxmax()
AM2_try = TS_.groupby(['yearAM'])['Q_S2'].idxmax()
AM3_try = TS_.groupby(['yearAM'])['Q_C'].idxmax()

In [ ]:
am1 = pd.DataFrame() #columns=['Time', 'year', 'AccumP_rol0', 'AccumP_rol1']
am2 = pd.DataFrame()
am3 = pd.DataFrame()
for i in range(0, 10000): #len(AM1_try)
    am1 = am1.append(TS_.loc[AM1_try[i]])#, ignore_index=True)
    am2 = am2.append(TS_.loc[AM2_try[i]])
    am3 = am3.append(TS_.loc[AM3_try[i]])
    
# for i in range(len(AM1_try)): #len(AM1_try)
#     am1 = am1.append(TS_.loc[AM1_try[i]])#, ignore_index=True)
#     am2 = am2.append(TS_.loc[AM2_try[i]])
#     am3 = am3.append(TS_.loc[AM3_try[i]])

In [ ]:
for i in range(10000, 20000): #len(AM1_try)
    am1 = am1.append(TS_.loc[AM1_try[i]])#, ignore_index=True)
    am2 = am2.append(TS_.loc[AM2_try[i]])
    am3 = am3.append(TS_.loc[AM3_try[i]])

In [ ]:
AM1_try['ID'] = TS_.groupby(['yearAM'])['Q_S1'].idxmax()

In [ ]:
am1_, am2_, am3_, savg_S1_, savg_S2_, savg_C_ = AM(TS)

## HID_03
### 1 file of AM
Only from 5 to 24

In [ ]:
def AddSavg(am1, S1_savg, S2_savg, C_savg):
    conditionsALL = [am1.season=='DJF', am1.season=='JJA', am1.season=='MAM', am1.season=='SON']

    choicesS1 = [S1_savg['DJF'], S1_savg['JJA'], S1_savg['MAM'], S1_savg['SON']]
    choicesS2 = [S2_savg['DJF'], S2_savg['JJA'], S2_savg['MAM'], S2_savg['SON']]
    choicesC = [C_savg['DJF'], C_savg['JJA'], C_savg['MAM'], C_savg['SON']]

    am1['S1savg'] = np.select(conditionsALL, choicesS1, default='ERROR').astype(float)
    am1['S2savg'] = np.select(conditionsALL, choicesS2, default='ERROR').astype(float)
    am1['Csavg'] = np.select(conditionsALL, choicesC, default='ERROR').astype(float)
    am1['S1/S1savg'] = am1.Q_S1 / am1.S1savg
    am1['S2/S2savg'] = am1.Q_S2 / am1.S2savg
    am1['C/Csavg'] = am1.Conf / am1.Csavg
    return am1

In [ ]:
count = 0
am1 = pd.DataFrame()
am2 = pd.DataFrame()
am3 = pd.DataFrame()
savg_S1 = 0
savg_S2 = 0
savg_C = 0


sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_40000peaks\C68_" 
# C68_40000peaks

for j in range(5, 25):  # 0, 25 
    op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_AM\C68_AM%d_" %j
#     op =  r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\02_AMdisc%d" %w + r"d\C%d_" %c + r"%d_" %j 
#     op =  r"P:\11206883-006-dar-cloud-computing\GRADE_export\HID\AMdisc%d" %w + r"d\C%d_" %c + r"%d_" %j 
    am1_ = pd.read_pickle(op + "AM1.pkl")
    am2_ = pd.read_pickle(op + "AM2.pkl")
    am3_ = pd.read_pickle(op + "AM3.pkl")
    savg_S1_ = pd.read_pickle(op + "savg_S1.pkl")
    savg_S2_ = pd.read_pickle(op + "savg_S2.pkl")
    savg_C_ = pd.read_pickle(op + "savg_C.pkl")
    am1 = pd.concat([am1, am1_])
    am2 = pd.concat([am2, am2_])
    am3 = pd.concat([am3, am3_])
    savg_S1 += savg_S1_
    savg_S2 += savg_S2_
    savg_C += savg_C_
    count += 1
#from all the loops, get am1, am2, am3, savg_S1, savg_S2, savg_C combinado de los 50,000 anos
S1_savg = savg_S1 / count
S2_savg = savg_S2 / count
C_savg = savg_C / count

AM1 = AddSavg(am1, S1_savg, S2_savg, C_savg)
AM2 = AddSavg(am2, S1_savg, S2_savg, C_savg)
AM3 = AddSavg(am3, S1_savg, S2_savg, C_savg)
#     now we want to save the 3 data frames per confluence
AM1.to_pickle(sv + "AM1.pkl")
AM2.to_pickle(sv + "AM2.pkl")
AM3.to_pickle(sv + "AM3.pkl")

In [ ]:
def plotSsets(AM1, AM2, AM3, c, td, unit, peaks, color):
#     AM1 = pd.read_pickle(op + "AM1.pkl")
#     AM2 = pd.read_pickle(op + "AM2.pkl")
#     AM3 = pd.read_pickle(op + "AM3.pkl")
    
    AM1['ind'] = np.arange(len(AM1))
    AM2['ind'] = np.arange(len(AM2))
    AM3['ind'] = np.arange(len(AM3))
    randomindex1 = np.random.choice(AM1.ind, size=peaks, replace=False)
    set1 = pd.DataFrame()
    set2 = pd.DataFrame()
    set3 = pd.DataFrame()
    for i in range(peaks):
        set1 = set1.append(AM1.loc[AM1.ind == randomindex1[i]])
        set2 = set2.append(AM2.loc[AM2.ind == randomindex1[i]])
        set3 = set3.append(AM3.loc[AM3.ind == randomindex1[i]])
    
    
    plt.figure(figsize=(7, 7))
    plt.plot(set1['Q_S1'], set1['Q_S2'], 'o', color='none', markeredgecolor=color,markersize=10, label='Set 1: S1max-S2conc')
    plt.plot(set2['Q_S1'], set2['Q_S2'], 'x', color=color, markersize=9, label='Set 2: S2max-S1conc')
    plt.plot(set3['Q_S1'], set3['Q_S2'], '.', color=color, markersize=10, label='Set 3: Cmax (C=S1+S2)')
    plt.title(f'{td} | C{c+1} ({peaks} peaks AM)', fontsize=20, y=1.05)
    plt.xlabel(f'S1 ({unit})', fontsize=18)
    plt.ylabel(f'S2 ({unit})', fontsize=18)
    plt.xlim(left=0, right=16000)
    plt.ylim(ymin = 0, ymax =4000)
    plt.xticks(fontsize=12, rotation=90)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")

    plt.grid()
    return

In [ ]:
color='blue'

plt.figure(figsize=(7, 7))
plt.plot(AM1['Q_S1'], AM1['Q_S2'], 'o', color='none', markeredgecolor=color,markersize=10, label='Set 1: S1max-S2conc')
plt.plot(AM2['Q_S1'], AM2['Q_S2'], 'x', color=color, markersize=9, label='Set 2: S2max-S1conc')
plt.plot(AM3['Q_S1'], AM3['Q_S2'], '.', color=color, markersize=10, label='Set 3: Cmax (C=S1+S2)')
# plt.title(f'{td} | C{c+1} ({peaks} peaks AM)', fontsize=20, y=1.05)
# plt.xlabel(f'S1 ({unit})', fontsize=18)
# plt.ylabel(f'S2 ({unit})', fontsize=18)
plt.xlim(left=0, right=10000)
plt.ylim(ymin = 0, ymax =5000)
plt.xticks(fontsize=12, rotation=90)
plt.yticks(fontsize=12)
plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")
plt.grid()

## Metrics

In [ ]:
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_40000peaks\C68_" 
AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

In [ ]:
AM1.index = np.arange(len(AM1))
AM2.index = np.arange(len(AM2))
AM3.index = np.arange(len(AM3))

In [ ]:
rho1, Sp1 = spearmanr(AM1['Q_S1'], AM1['Q_S2'])
rho2, Sp2 = spearmanr(AM2['Q_S1'], AM2['Q_S2'])
rho3, Sp3 = spearmanr(AM3['Q_S1'], AM3['Q_S2'])
print(f'AM1_rho={rho1:.4}, AM2_rho={rho2:.4}, AM3_rho={rho3:.4}')

In [ ]:
rho1, Sp1 = spearmanr(am1_['Q_S1'], am1_['Q_S2'])
rho2, Sp2 = spearmanr(am2_['Q_S1'], am2_['Q_S2'])
rho3, Sp3 = spearmanr(am3_['Q_S1'], am3_['Q_S2'])
print(f'AM1_rho={rho1:.4}, AM2_rho={rho2:.4}, AM3_rho={rho3:.4}')

In [ ]:
AM1m = AM1['Q_S1'].min()
AM2m = AM2['Q_S1'].min()
AM3m = AM3['Q_S1'].min()
print(f'AM1_S1min={AM1m:.2f}, AM2_S1min={AM2m:.2f}, AM3_S1min={AM3m:.2f}')
AM1m = AM1['Q_S1'].max()
AM2m = AM2['Q_S1'].max()
AM3m = AM3['Q_S1'].max()
print(f'AM1_S1max={AM1m:.2f}, AM2_S1max={AM2m:.2f}, AM3_S1max={AM3m:.2f}')
AM1m = AM1['Q_S2'].min()
AM2m = AM2['Q_S2'].min()
AM3m = AM3['Q_S2'].min()
print(f'AM1_S2min={AM1m:.2f}, AM2_S2min={AM2m:.2f}, AM3_S2min={AM3m:.2f}')
AM1m = AM1['Q_S2'].max()
AM2m = AM2['Q_S2'].max()
AM3m = AM3['Q_S2'].max()
print(f'AM1_S2max={AM1m:.2f}, AM2_S2max={AM2m:.2f}, AM3_S2max={AM3m:.2f}')

In [ ]:
AM1[(AM1.month == 1)&((AM1.day == 1)|(AM1.day == 2)|(AM1.day == 3)|(AM1.day == 4)|(AM1.day == 5)|(AM1.day == 6)|(AM1.day == 7))]
AM1[(AM1.month == 12)&((AM1.day == 31)|(AM1.day == 30)|(AM1.day == 29)|(AM1.day == 28)|(AM1.day == 27)|(AM1.day == 26)|(AM1.day == 25))]
AM2[(AM2.month == 1)&((AM2.day == 1)|(AM2.day == 2)|(AM2.day == 3)|(AM2.day == 4)|(AM2.day == 5)|(AM2.day == 6)|(AM2.day == 7))]
AM2[(AM2.month == 12)&((AM2.day == 31)|(AM2.day == 30)|(AM2.day == 29)|(AM2.day == 28)|(AM2.day == 27)|(AM2.day == 26)|(AM2.day == 25))]
# AM2[AM2.month == 1]
AM3[(AM3.month == 1)&((AM3.day == 1)|(AM3.day == 2)|(AM3.day == 3)|(AM3.day == 4)|(AM3.day == 5)|(AM3.day == 6)|(AM3.day == 7))]
AM3[(AM3.month == 12)&((AM3.day == 31)|(AM3.day == 30)|(AM3.day == 29)|(AM3.day == 28)|(AM3.day == 27)|(AM3.day == 26)|(AM3.day == 25))]
# AM3[AM3.month == 1]

In [ ]:
for i in range(1, 13):
#     print(f'{i} - AM1={len(AM1[AM1.month == i])} - AM2={len(AM2[AM2.month == i])} - AM3={len(AM3[AM3.month == i])}')
    print(f'{i} - AM1={len(AM1[AM1.month == i])/39960*100:.2f} - AM2={len(AM2[AM2.month == i])/39960*100:.2f} - AM3={len(AM3[AM3.month == i])/39960*100:.2f}')

In [ ]:
AM1_hist = []
AM2_hist = []
AM3_hist = []

for i in range(1, 13):
    AM1_hist.append(len(AM1_C68[AM1_C68.month == i])) 
    AM2_hist.append(len(AM2_C68[AM2_C68.month == i])) 
    AM3_hist.append(len(AM3_C68[AM3_C68.month == i])) 
#     globals()[f'AM1_m{i}'] = 
#     globals()[f'AM2_m{i}'] = 
#     globals()[f'AM3_m{i}'] = 

#     print(f'{i} - AM1={} - AM2={len(AM2_C68[AM2_C68.month == i])} - AM3={len(AM3_C68[AM3_C68.month == i])}')
    
    
    print(f'{i} - AM1={len(AM1_C68[AM1_C68.month == i])} - AM2={len(AM2_C68[AM2_C68.month == i])} - AM3={len(AM3_C68[AM3_C68.month == i])}')
#     print(f'{i} - AM1={len(AM1_C68[AM1_C68.month == i])/49960*100:.2f} - AM2={len(AM2_C68[AM2_C68.month == i])/49960*100:.2f} - AM3={len(AM3_C68[AM3_C68.month == i])/49960*100:.2f}')

AM1_hist, AM2_hist, AM3_hist

In [ ]:
AM1m = AM1['Q_S1'].argmin()
AM2m = AM2['Q_S1'].argmin()
AM3m = AM3['Q_S1'].argmin()
print(f'AM1_S1min={AM1m:.0f}, AM2_S1min={AM2m:.0f}, AM3_S1min={AM3m:.0f}')
AM1m = AM1['Q_S1'].argmax()
AM2m = AM2['Q_S1'].argmax()
AM3m = AM3['Q_S1'].argmax()
print(f'AM1_S1max={AM1m:.0f}, AM2_S1max={AM2m:.0f}, AM3_S1max={AM3m:.0f}')
AM1m = AM1['Q_S2'].argmin()
AM2m = AM2['Q_S2'].argmin()
AM3m = AM3['Q_S2'].argmin()
print(f'AM1_S2min={AM1m:.0f}, AM2_S2min={AM2m:.0f}, AM3_S2min={AM3m:.0f}')
AM1m = AM1['Q_S2'].argmax()
AM2m = AM2['Q_S2'].argmax()
AM3m = AM3['Q_S2'].argmax()
print(f'AM1_S2max={AM1m:.0f}, AM2_S2max={AM2m:.0f}, AM3_S2max={AM3m:.0f}')

### Time series of the peaks (min&max)

In [ ]:
for j in range(5, 25):
    globals()[f'C68_{j}'] = pd.read_csv(r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\C68_TS\C68_TS%d.csv" %j, delimiter=';')


In [ ]:
def plotAM(self, b1, b2):
        """
        Creates two graphs: 
            Annual maximas and concurrent flows of the TWO sets
            Discharge of TWO points with their annual maxima and concurrent flow
        b1, b2: Locations ('Q_#Point1', 'Q_#Point2')
        Returns: plot
        """
        self.b1 = b1
        self.b2 = b2
        s1, s2, am1, am2 = self.AnnualMax(self.b1, self.b2)
        plt.figure(figsize=(15, 5))
        plt.plot(am1['Year'], am1[self.b1], 'b-o', label=f'{self.b1} Maximun')
        plt.plot(am2['Year'], am2[self.b1], 'b--o', label=f'{self.b1}  Concurrent')
        plt.plot(am2['Year'], am2[self.b2], 'g-o', label=f'{self.b2} Maximun')
        plt.plot(am1['Year'], am1[self.b2], 'g--o', label=f'{self.b2} Concurrent')
        plt.title('Annual Maxima')
        plt.xlabel('Date')
        plt.ylabel('Discharge (m3/sec)') 
        plt.legend()
        plt.grid()
        self.plotQ(self.b1, self.b2)   
        plt.plot(am1['time'], am1[self.b1], color='midnightblue', marker='o', ls='', label=f'{b1} Maximun')
        plt.plot(am2['time'], am2[self.b1], color='midnightblue', marker='x', ls='', label=f'{b1}  Concurrent')
        plt.plot(am2['time'], am2[self.b2], color='darkolivegreen', marker='o', ls='', label=f'{b2} Maximun')
        plt.plot(am1['time'], am1[self.b2], color='darkolivegreen', marker='x', ls='', label=f'{b2} Concurrent')
        plt.title('Annual Maxima')
        plt.xlabel('Date')
        plt.ylabel('Discharge (m3/sec)') 
        plt.legend()
        return

In [ ]:
Year = 46560
arg = 34559
plt.figure(figsize=(20, 5))
plt.plot(C68_22.Q_S1[C68_22.year == Year], 'b', label='Q_S1')
plt.plot(C68_22.Q_S2[C68_22.year == Year], 'g', label='Q_S2')
#the maxima
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S1')], color='midnightblue', marker='o', ls='', label=f'Q_S1 Maximun')
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='x', ls='', label=f'Q_S2 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S1')], color='midnightblue', marker='x', ls='', label=f'Q_S1 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='o', ls='', label=f'Q_S2 Maximun')
plt.axhline(y = np.mean(C68_22.Q_S1), color = 'r', linestyle = '--', label=f'S1_mean')
plt.title('S1_max AM1') #Runoff
plt.xlabel('Date')
plt.ylabel('Discharge (m3/sec)') # Q m3/s
plt.legend()
plt.grid()

In [ ]:
Year = 27654
arg = 15653
plt.figure(figsize=(20, 5))
plt.plot(C68_12.Q_S1[C68_12.year == Year], 'b', label='Q_S1')
plt.plot(C68_12.Q_S2[C68_12.year == Year], 'g', label='Q_S2')
# plt.plot(C68_24.Q_S1[37619:38349], 'b', label='Q_S1') #37900:38000
# plt.plot(C68_24.Q_S2[37619:38349], 'g', label='Q_S2')
# 37985
#the maxima
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S1')], color='midnightblue', marker='o', ls='', label=f'Q_S1 Maximun')
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='x', ls='', label=f'Q_S2 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S1')], color='midnightblue', marker='x', ls='', label=f'Q_S1 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='o', ls='', label=f'Q_S2 Maximun')
plt.axhline(y = np.mean(C68_12.Q_S1), color = 'r', linestyle = '--', label=f'S1_mean')
plt.title('S1_min AM2') #Runoff
plt.xlabel('Date')
plt.ylabel('Discharge (m3/sec)') # Q m3/s
plt.legend()
plt.grid()

In [ ]:
# C68_24.loc[C68_24.year == 50104]

In [ ]:
Year = 18748
arg = 6747
plt.figure(figsize=(20, 5))
plt.plot(C68_8.Q_S1[C68_8.year == Year], 'b', label='Q_S1')
plt.plot(C68_8.Q_S2[C68_8.year == Year], 'g', label='Q_S2')
#the maxima
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S1')], color='midnightblue', marker='o', ls='', label=f'Q_S1 Maximun')
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='x', ls='', label=f'Q_S2 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S1')], color='midnightblue', marker='x', ls='', label=f'Q_S1 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='o', ls='', label=f'Q_S2 Maximun')
plt.axhline(y = np.mean(C68_8.Q_S1), color = 'r', linestyle = '--', label=f'S1_mean')
plt.title('S1_min (2) AM1') #Runoff
plt.xlabel('Date')
plt.ylabel('Discharge (m3/sec)') # Q m3/s
plt.legend()
plt.grid()

In [ ]:
Year = 51034
arg = 39033
plt.figure(figsize=(20, 5))
# plt.plot(C68_24.Q_S1[C68_24.year == 51033], 'b', label='Q_S1')
# plt.plot(C68_24.Q_S2[C68_24.year == 51033], 'g', label='Q_S2')
plt.plot(C68_24.Q_S1[377200:377500], 'b', label='Q_S1')
plt.plot(C68_24.Q_S2[377200:377500], 'g', label='Q_S2')
#the maxima
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S1')], color='midnightblue', marker='o', ls='', label=f'Q_S1 Maximun')
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='x', ls='', label=f'Q_S2 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S1')], color='midnightblue', marker='x', ls='', label=f'Q_S1 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='o', ls='', label=f'Q_S2 Maximun')
plt.axhline(y = np.mean(C68_24.Q_S2), color = 'r', linestyle = '--', label=f'S2_mean')
plt.title('S2_max AMx') #Runoff
plt.xlabel('Date')
plt.ylabel('Discharge (m3/sec)') # Q m3/s
plt.legend()
plt.grid()

In [ ]:
C68_24.Q_S1[C68_24.year == 51034]

In [ ]:
Year = 37174
arg = 25173
plt.figure(figsize=(20, 5))
plt.plot(C68_17.Q_S1[C68_17.year == Year], 'b', label='Q_S1')
plt.plot(C68_17.Q_S2[C68_17.year == Year], 'g', label='Q_S2')
#the maxima
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S1')], color='midnightblue', marker='o', ls='', label=f'Q_S1 Maximun')
plt.plot(AM1.loc[(arg, 'Unnamed: 0')], AM1.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='x', ls='', label=f'Q_S2 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S1')], color='midnightblue', marker='x', ls='', label=f'Q_S1 Concurrent')
plt.plot(AM2.loc[(arg, 'Unnamed: 0')], AM2.loc[(arg, 'Q_S2')], color='darkolivegreen', marker='o', ls='', label=f'Q_S2 Maximun')
plt.axhline(y = np.mean(C68_17.Q_S2), color = 'r', linestyle = '--', label=f'S2_mean')
plt.title('S2_min AM1') #Runoff
plt.xlabel('Date')
plt.ylabel('Discharge (m3/sec)') # Q m3/s
plt.legend()
plt.grid()